## E05 - Capacidade de Processos

In [1]:
library(googlesheets4)
library(qcc)

Warning message:
"package 'googlesheets4' was built under R version 4.3.3"
Warning message:
"package 'qcc' was built under R version 4.3.3"
Package 'qcc' version 2.7

Type 'citation("qcc")' for citing this R package in publications.



In [2]:
url_planilha <- "https://docs.google.com/spreadsheets/d/1R0C885chVDklaK92j19qcCUZHnGzNtmLs7sYlQOmtd4/edit?gid=0#gid=0"
gs4_deauth()
# respostas_formulario <- read_sheet(url_planilha)

In [3]:
dados_vazao <- read_sheet(url_planilha, sheet = 2)
dados_niveis <- read_sheet(url_planilha, sheet = 3)
temp_quente <- read_sheet(url_planilha, sheet = 4)
temp_c <- read_sheet(url_planilha, sheet = 5)
temp_frio <- read_sheet(url_planilha, sheet = 6)

✔ Reading from Controle Estatístico de Processos - Variáveis.

✔ Range ''vazao''.

✔ Reading from Controle Estatístico de Processos - Variáveis.

✔ Range ''nivel''.

✔ Reading from Controle Estatístico de Processos - Variáveis.

✔ Range ''tempQ''.

✔ Reading from Controle Estatístico de Processos - Variáveis.

✔ Range ''tempC''.

✔ Reading from Controle Estatístico de Processos - Variáveis.

✔ Range ''tempF''.



### a) Estatísticas Básicas

In [7]:
Q <- dados_vazao$`Q (L/min)`  

# ---- (1) Estatísticas básicas
n <- length(Q)
x_bar <- mean(Q)
sigma_LT <- sd(Q)  # σLT de longo prazo

n
x_bar
sigma_LT

[1] 4

[1] 1.044421

[1] 0.1212645

### b) Especificações e Metas

In [8]:
LSL <- 0.9
USL <- 1.1
T <- 1.0
delta_c <- (USL - LSL) / 2

### c) Desempenho

In [10]:
Pp <- (USL - LSL) / (6 * sigma_LT)
PpkU <- (USL - x_bar) / (3 * sigma_LT)
PpkL <- (x_bar - LSL) / (3 * sigma_LT)
Ppk <- min(PpkU, PpkL)

Pp 
PpkU 
PpkL 
Ppk 

[1] 0.2748813

[1] 0.1527775

[1] 0.3969852

[1] 0.1527775

### d)

In [12]:
Q_matrix <- matrix(Q, ncol = 3, byrow = TRUE)
R_i <- apply(Q_matrix, 1, function(x) max(x) - min(x))
R_bar <- mean(R_i)
d2 <- 1.693  # para n = 3
sigma_ST <- R_bar / d2

Q_matrix 
R_i
R_bar 
d2 
sigma_ST 

Warning message in matrix(Q, ncol = 3, byrow = TRUE):
"comprimento dos dados [4] não é um submúltiplo ou múltiplo do número de colunas [3]"


0.9661836,1.0169492,0.9714726
1.2230769,0.9661836,1.0169492


[1] 0.05076558 0.25689335

[1] 0.1538295

[1] 1.693

[1] 0.09086206

### e) em diante

In [13]:
Cp <- (USL - LSL) / (6 * sigma_ST)
CpkU <- (USL - x_bar) / (3 * sigma_ST)
CpkL <- (x_bar - LSL) / (3 * sigma_ST)
Cpk <- min(CpkU, CpkL)

# ---- (6) Cpmk com desvio do alvo
Cpmk <- min((USL - T), (T - LSL)) / (3 * sqrt(sigma_ST^2 + (x_bar - T)^2))

# ---- (7) Função de perda de Taguchi
Lc <- 1.00  # custo do consumidor (R$/litro)
Lm <- 0.05  # custo do produtor (R$/litro)
k_c <- Lc / delta_c^2
delta_m <- delta_c * sqrt(Lm / Lc)
k_m <- Lm / delta_m^2

# L(x) consumidor e produtor
L_taguchi_c <- mean(k_c * (Q - T)^2)
L_taguchi_m <- mean(k_m * (Q - T)^2)

In [14]:
cat("🔧 Estatísticas da Vazão\n")
cat("Média:", round(x_bar, 4), " | σLT:", round(sigma_LT, 4), " | σST:", round(sigma_ST, 4), "\n\n")

cat("📏 Índices de desempenho:\n")
cat("Pp:", round(Pp, 3), "| PpkU:", round(PpkU, 3), "| PpkL:", round(PpkL, 3), "| Ppk:", round(Ppk, 3), "\n\n")

cat("🏭 Índices de capacidade:\n")
cat("Cp:", round(Cp, 3), "| Cpk:", round(Cpk, 3), "| Cpmk:", round(Cpmk, 3), "\n\n")

cat("📉 Função de perda de Taguchi:\n")
cat("VoC (consumidor): R$", round(L_taguchi_c, 4), "\n")
cat("VoP (produtor):   R$", round(L_taguchi_m, 4), "\n")

🔧 Estatísticas da Vazão
Média: 1.0444  | σLT: 0.1213  | σST: 0.0909 

📏 Índices de desempenho:
Pp: 0.275 | PpkU: 0.153 | PpkL: 0.397 | Ppk: 0.153 

🏭 Índices de capacidade:
Cp: 0.367 | Cpk: 0.204 | Cpmk: 0.33 

📉 Função de perda de Taguchi:
VoC (consumidor): R$ 1.3002 
VoP (produtor):   R$ 1.3002 
